# Building a Python Web Scraping Project From Scratch


**Steps:**
- Scrape https://github.com/topics
- Get a list of all topics. For each topic, get topic title, topic page URL and topic description
- For each topic, Get the top 25 repositories in the topic from the topic page
- For each repository, Grab the repo name, username, stars and repo URL
- At last create a CSV file by compling all scraped data

## Scraping Popular GitHub Topics
- `requests` to downlaod the page
- `BeautifulSoup` to parse and extract information
- `Pandas` to convert into dataframe

Format of Final Topics Dataframe
Attributes:
1. `title` - Name of the topic - [3D]

2. `description` - Description of that topics - [3D modeling uses specialized software to create a digital model of a physical object. It is an aspect of 3D computer graphics, used for video games, 3D printing, and VR, among other applications.]
3. `url` - URL of that topic - [https://github.com/topics/3d]

#### Importing required libraries

In [155]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Function to Get Topic Page from Github Using `requests` Library

In [156]:
def get_topics_page(topics_url):
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topics_url))
    page = BeautifulSoup(response.text, 'html.parser')
    return page

#### Function to Get Popular Topic's Title

In [157]:
def get_topic_titles(page):
    topic_title_tags = page.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text.strip())
    return topic_titles

#### Function to Get Topic's Discription

In [158]:
def get_topic_descs(page):
    topic_desc_tags = page.find_all('p', {'class': 'f5 color-text-secondary mb-0 mt-1'})
    topic_descs = []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs

#### Function to Get Topic's URL

In [159]:
def get_topic_urls(page):
    topic_link_tags = page.find_all('a', {'class': 'd-flex no-underline'})
    topic_urls = []
    for tag in topic_link_tags:
        topic_urls.append('https://github.com' + tag['href'])
    return topic_urls

#### Scraping all Data and Formating It Into CSV File

In [160]:
def scrape_topics(topics_url):
    page = get_topics_page(topics_url)
    topics_dict = {
        'title': get_topic_titles(page),
        'description': get_topic_descs(page),
        'url': get_topic_urls(page)
    }
    return pd.DataFrame(topics_dict)

##### Scraping All Existing Topic Pages and Compile Them Into Single CSV File

In [161]:
pages = [1, 2, 3, 4, 5, 6, 7]
topics_df = pd.DataFrame()
for i in pages:
    topics = scrape_topics("https://github.com/topics?page="+str(i))
    topics_df = topics_df.append(topics, ignore_index = True)

In [162]:
topics_df[:10]

,title,description,url
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency framework fo...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android
5,Angular,Angular is an open source web application plat...,https://github.com/topics/angular
6,Ansible,Ansible is a simple and powerful automation en...,https://github.com/topics/ansible
7,API,An API (Application Programming Interface) is ...,https://github.com/topics/api
8,Arduino,Arduino is an open source hardware and softwar...,https://github.com/topics/arduino
9,ASP.NET,ASP.NET is a web framework for building modern...,https://github.com/topics/aspnet


In [163]:
len(topics_df)

181



---



## Get the Top 25 Repositories Under Each Topic
For each repository, we'll grab the repo name, username, stars and repo URL
Format of Final Topics Dataframe Attributes:
- `repo_name` - Name of the repository
- `username` - Owner of that repository
- `stars` - Stars on that repository
- `repo_url` - URL of that repository